In [4]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import glob
import torch.nn as nn
import os.path as osp
import torchvision
import matplotlib.pyplot as plt
import torch
#import rotateImage
import cv2
import numpy as np
#import split_folders
import random
import torch.backends.cudnn as cudnn
import torch.optim as optim
from random import choice
from collections import OrderedDict

In [5]:
#/content/Car-damage/carPartModel-305.pth
#/content/drive/My Drive/Car-part-damage/carPartModel-305.pth
model1 = torch.load('/content/drive/My Drive/Car-part-damage/carPartModel-305.pth',map_location=torch.device('cpu'))
#model1 = torch.load('/content/Car-damage/carPartModel-305.pth')
for names in model1:
  print(names)
#for name,param in model1.items():
#   if name=='state_dict':
#     print(param)

epoch
state_dict
class_to_idx
confusion_matrix


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#from PIL import Image, ImageFile
#img = Image.open("/content/drive/My Drive/IMG-20200729-WA0036.jpg")


In [ ]:
#!unzip "/content/drive/My Drive/carDamageDetection.zip" -d '/content/drive/My Drive/carDamageDetection'

In [6]:
def load_model(checkpoint_path):
  chpt = torch.load(checkpoint_path)
  net = torchvision.models.resnet50(pretrained=True)
  fc = nn.Sequential(OrderedDict([
    ("dropout", nn.Dropout(0.4)),
    ("out", nn.Linear(2048, 11))]))
  net.fc = fc

  for param, x in net.named_parameters():
    if param.startswith("fc.out"):
        x.requires_grad = True
    else:
        x.requires_grad = False

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  #net.class_to_idx = classes
  net = net.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
    
  net.class_to_idx = chpt['class_to_idx']       
  net.load_state_dict(chpt['state_dict'])  
  return net

In [7]:
the_model = load_model('/content/drive/My Drive/Car-part-damage/carPartModel-305.pth')
# /content/drive/My Drive/carDamageDetection.zip

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
dummy_input = torch.randn(1, 3, 224, 224, device='cuda')
onnx_path =  "/content/pytorch_ONNx.onnx"

torch.onnx.export(the_model, dummy_input, onnx_path, verbose=False)
print('Model exported to ' + onnx_path)

Model exported to /content/pytorch_ONNx.onnx


In [8]:
dummy_input = torch.randn(1, 3, 224, 224, device='cuda')

In [9]:
#PyTorch Profiler
import torch.autograd.profiler as profiler

In [10]:
with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        the_model(dummy_input)

In [11]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))

---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                         Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     Number of Calls  
---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
model_inference              19.35%           24.550ms         100.00%          126.858ms        126.858ms        1                
adaptive_avg_pool2d          0.03%            39.286us         23.11%           29.320ms         29.320ms         1                
mean                         23.05%           29.239ms         23.06%           29.256ms         29.256ms         1                
conv2d                       6.21%            7.881ms          19.76%           25.070ms         473.012us        53               
convolution                  0.22%            280.755us        13.55%      

In [14]:
with profiler.profile(profile_memory=True, record_shapes=True) as prof:
    the_model(dummy_input)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=20))

---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                         Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     CPU Mem          Self CPU Mem     CUDA Mem         Self CUDA Mem    Number of Calls  
---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
conv2d                       1.51%            322.809us        43.43%           9.277ms          175.045us        0 b              0 b              43.33 Mb         0 b              53               
convolution                  1.22%            260.408us        41.91%           8.955ms          168.955us        0 b              0 b              43.33 Mb         0 b              53               

In [15]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=20))

---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                         Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     CPU Mem          Self CPU Mem     CUDA Mem         Self CUDA Mem    Number of Calls  
---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
conv2d                       1.51%            322.809us        43.43%           9.277ms          175.045us        0 b              0 b              43.33 Mb         0 b              53               
convolution                  1.22%            260.408us        41.91%           8.955ms          168.955us        0 b              0 b              43.33 Mb         0 b              53               

In [12]:
#!unzip '/content/Car-damage/splitTrainValnew.zip' -d '/content/Car-damage/splitTrainValnew' 

In [ ]:
!pip install split-folders

In [ ]:
!pip install split-folders tqdm

In [ ]:
import splitfolders
#import split_folders

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import glob
import torch.nn as nn
import os.path as osp
import torchvision
import matplotlib.pyplot as plt
import torch
#import rotateImage
import cv2
import numpy as np
#import split_folders
import splitfolders
import random
import torch.backends.cudnn as cudnn
import torch.optim as optim
from random import choice
from collections import OrderedDict

In [ ]:
dataPath = "/content/Car-damage/splitTrainValnew"
splitfolders.ratio(dataPath, output="/content/Car-damage/splitTrainValnewout/", seed=1337, ratio=(.85, .05, .1))


Copying files: 0 files [00:00, ? files/s]


In [ ]:
class CarPartDataset(Dataset):
    def __init__(self, root,):
        self.filenames, self.labels = [], []
        self.root = root
        self.temp = 0
        self.images = 0
        self.len = 0
        self.imagesList = []
        filenames = []
        filenames = glob.glob(osp.join(self.root, '*/*.*'))

        if (self.root.endswith("/train")):
            for fn in filenames:
                if ".tif" not in fn:
                    self.filenames.append(fn)
                    self.labels.append(osp.dirname(fn).split("/content/Car-damage/car-train-val/splitTrainVal/train")[1])
        elif (self.root.endswith("/val")):
            for fn in filenames:
                if ".tif" not in fn:
                    self.filenames.append(fn)
                    self.labels.append(osp.dirname(fn).split("/content/Car-damage/car-train-val/splitTrainVal/val")[1])

        self.len = len(self.filenames)
        self.label2idx = {l: i for i, l in enumerate(list(set(self.labels)))}

    def className(self):
        return self.label2idx

    # flipping an image
    def flipping(self, imageForAug2):
        imageForAug2 = cv2.flip(imageForAug2, flipCode=1)
        return imageForAug2

    # rotating an image at an angle  randomly chosen between -20 to 20
    def rotation(self, imageForAug3):
        angle = random.randint(-20,20)
        imageForAug3 = rotateImage.rotate_im(imageForAug3,angle)
        return imageForAug3

    def augmentedData(self, imageName):
        try:
            image = cv2.imread(imageName)
            img = cv2.resize(image, (244, 244), interpolation=cv2.INTER_AREA)
            methodsList = ["flipping", "rotation"]
            thres = 0.2
            temp = np.random.randn()
            if(temp>thres):
                x = list(set(random.choices(methodsList, k=random.randint(1,3))))
                # print(x)
                for i in x:
                    if(i == "flipping" ):
                        img = self.flipping(img)
                    elif (i == "rotation"):
                        img = self.rotation(img)
            else:
               img = img
            return img
        except Exception:
            print(imageName)

    def __getitem__(self, index):
        try:
            if "/train/" in self.filenames[index]:
                image = self.augmentedData(self.filenames[index])
#                 print(self.filenames[index])
                image = torch.tensor(image).permute((2, 0, 1))
                label = self.labels[index]
                return image, self.label2idx[label]
            elif "/val/" in self.filenames[index]:
                image = cv2.imread(self.filenames[index])
#                 print(self.filenames[index])
                image = cv2.resize(image, (244, 244), interpolation=cv2.INTER_AREA)
                image = torch.tensor(image).permute((2, 0, 1))
                label = self.labels[index]
                return image, self.label2idx[label]
        except Exception:
            print(self.filenames[index])

    def __len__(self):
        return self.len

In [ ]:
#TrainSetPath = "./splitTrainVal/train"
#/content/Car-damage/car-train-val/splitTrainVal
#/content/Car-damage/car-train-val/splitTrainVal/train
TrainSetPath = "/content/Car-damage/car-train-val/splitTrainVal/train"
#ValSetPath = "./splitTrainVal/val"
ValSetPath = "/content/Car-damage/car-train-val/splitTrainVal/val"
train_datasets = CarPartDataset(TrainSetPath)
valid_datasets = CarPartDataset(ValSetPath)
print(len(train_datasets))
print(len(valid_datasets))
train_loader = DataLoader(train_datasets, batch_size=64, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_datasets, batch_size=64, shuffle=True, num_workers=2)
print(len(train_loader))

1170
161
19


In [ ]:
# functions to show an image
def imshow(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.imshow(inp)
    plt.show()

images, labels = next(iter(train_loader))

temp1 = []

obj = CarPartDataset(TrainSetPath)
classes = CarPartDataset.className(obj)
# print(classes)
for key, value in classes.items():
    for z in range(0, 64):
        if value == labels[z]:
            temp1.append(key)

image_grid = torchvision.utils.make_grid(images)
imshow(image_grid)
# print(temp1)

print("Building Model")
net = torchvision.models.resnet50(pretrained=True)
fc = nn.Sequential(OrderedDict([
    ("dropout", nn.Dropout(0.4)),
    ("out", nn.Linear(2048, 11))]))
net.fc = fc

for param, x in net.named_parameters():
    if param.startswith("fc.out"):
        x.requires_grad = True
    else:
        x.requires_grad = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.class_to_idx = classes
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
plt_train_loss, plt_val_loss = [], []
plt_batch_train_loss, plt_batch_val_loss = [],[]

global prev_loss
prev_loss = float("inf")
global count
count = 0

# training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    L1 = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        L1 += loss.item()*inputs.size(0)
        loss.backward()
        optimizer.step()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
#         print('Batch Train Loss: %.3f | Acc: %.3f' % (L1/total, 100.*correct/total))
        plt_batch_train_loss.append(L1/total)

    epoch_loss = L1 / len(train_datasets)
    epoch_acc = correct / len(train_datasets)
    plt_train_loss.append(epoch_loss)
    print('{} Loss: {:.4f} Acc: {:.4f}'.format('Train', epoch_loss, epoch_acc))

# validation
def validation(epoch, flag):
    global prev_loss
    global count
    net.eval()
    valid_loss = 0
    valid_correct = 0
    total = 0
    L2 = 0
    xy = len(classes)
    conf_matrix = torch.zeros(xy, xy)

    for valid_idx ,(valid, y_valid) in enumerate(iter(valid_loader)):
        valid, y_valid = valid.to(device, dtype=torch.float), y_valid.to(device)
        output = net.forward(valid)
        L2 += criterion(output, y_valid).item()*valid.size(0)
        equal = (output.max(dim=1)[1] == y_valid.data)
        _, predd = output.max(1)
        valid_correct += torch.sum(equal)
        total += valid.size(0)
#         print('Batch Valid Loss: %.3f | Acc: %.3f' % (L2 / total, 100.*valid_correct / total))
        plt_batch_val_loss.append(L2 / total)
        for i in range(len(y_valid)):
            conf_matrix[predd[i], y_valid[i]] = conf_matrix[predd[i], y_valid[i]] + 1

    epoch_loss = L2 / len(valid_datasets)
    epoch_acc = valid_correct.double() / len(valid_datasets)

    print('{} Loss: {:.4f} Acc: {:.4f}'.format('Validation', epoch_loss, epoch_acc))

    plt_val_loss.append(epoch_loss)
    is_best = bool(epoch_loss < prev_loss)
    is_lessequal = bool(epoch_loss >= prev_loss)
    if (is_best):
        prev_loss = epoch_loss
        count = 0
    elif (is_lessequal):
        if (count < 25):
            count = count + 1
        else:
            print("Terminated Training for Early Stopping at Epoch ", epoch+1)
            flag = True

    if flag == False:
#         print(conf_matrix)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': net.state_dict(),
            'class_to_idx': net.class_to_idx,
            'confusion_matrix': conf_matrix
        }, is_best,"./models/carPartModel-"+epoch+".pth")
        # save_checkpoint(net.state_dict(),is_best)
    return flag

# save_checkpoint
def save_checkpoint(state, is_best, filename):
    if is_best:
        print(state["confusion_matrix"])
        print("=> Saving a new best")
        torch.save(state, filename)
    else:
        print("=> Validation loss did not reduce")

for epoch in range(0, 100):
    flag = False
    train(epoch)
    a = validation(epoch, flag)
    if a == True:
        break

plt.plot(plt_train_loss, label='Training loss')
plt.plot(plt_val_loss, label='Validation loss')
plt.legend(frameon=False)
plt.show()
x = np.arange(0, len(plt_batch_train_loss), len(plt_batch_train_loss)/len(plt_batch_val_loss))
plt.plot(plt_batch_train_loss,label = "Batch train loss")
plt.plot(x,plt_batch_val_loss, label = "Batch validation loss")
plt.legend(frameon=False)
plt.show()


/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-108.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/multipleDamage/multipleDamage-127.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/multipleDamage/multipleDamage-127.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterFront/glassShatterFront-16.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-108.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterFront/glassShatterFront-16.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/tailLamp/tailLamp-70.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/tailLamp/tailLamp-70.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterFront/glassShatterFront-103.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterFront/glassShatterFront-103.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/headLamp/headLamp-154.jpg
/content/Car-damage/ca

TypeError: ignored

/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-100.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-100.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/bumperScratch/bumperScratch-119.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/bumperScratch/bumperScratch-119.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-126.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-126.jpeg


In [ ]:

# functions to show an image
def imshow(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.imshow(inp)
    plt.show()

images, labels = next(iter(train_loader))

temp1 = []

obj = CarPartDataset(TrainSetPath)
classes = CarPartDataset.className(obj)
print(classes)
for key, value in classes.items():
    for z in range(0, 64):
        if value == labels[z]:
            temp1.append(key)

image_grid = torchvision.utils.make_grid(images)
imshow(image_grid)
# print(temp1)

print("Building Model")
net = torchvision.models.resnet50(pretrained=True)
fc = nn.Sequential(OrderedDict([
    ("dropout", nn.Dropout(0.4)),
    ("out", nn.Linear(2048, 11))]))
net.fc = fc

for param, x in net.named_parameters():
    if param.startswith("fc.out"):
        x.requires_grad = True
    else:
        x.requires_grad = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.class_to_idx = classes
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
plt_train_loss, plt_val_loss = [], []
plt_batch_train_loss, plt_batch_val_loss = [],[]

global prev_loss
prev_loss = float("inf")
global count
count = 0

# training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    L1 = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        L1 += loss.item()*inputs.size(0)
        loss.backward()
        optimizer.step()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
#         print('Batch Train Loss: %.3f | Acc: %.3f' % (L1/total, 100.*correct/total))
        plt_batch_train_loss.append(L1/total)

    epoch_loss = L1 / len(train_datasets)
    epoch_acc = correct / len(train_datasets)
    plt_train_loss.append(epoch_loss)
    print('{} Loss: {:.4f} Acc: {:.4f}'.format('Train', epoch_loss, epoch_acc))

# validation
def validation(epoch, flag):
    global prev_loss
    global count
    net.eval()
    valid_loss = 0
    valid_correct = 0
    total = 0
    L2 = 0
    xy = len(classes)
    conf_matrix = torch.zeros(xy, xy)

    for valid_idx ,(valid, y_valid) in enumerate(iter(valid_loader)):
        valid, y_valid = valid.to(device, dtype=torch.float), y_valid.to(device)
        output = net.forward(valid)
        L2 += criterion(output, y_valid).item()*valid.size(0)
        equal = (output.max(dim=1)[1] == y_valid.data)
        _, predd = output.max(1)
        valid_correct += torch.sum(equal)
        total += valid.size(0)
#         print('Batch Valid Loss: %.3f | Acc: %.3f' % (L2 / total, 100.*valid_correct / total))
        plt_batch_val_loss.append(L2 / total)
        for i in range(len(y_valid)):
            conf_matrix[predd[i], y_valid[i]] = conf_matrix[predd[i], y_valid[i]] + 1

    epoch_loss = L2 / len(valid_datasets)
    epoch_acc = valid_correct.double() / len(valid_datasets)

    print('{} Loss: {:.4f} Acc: {:.4f}'.format('Validation', epoch_loss, epoch_acc))

    plt_val_loss.append(epoch_loss)
    is_best = bool(epoch_loss < prev_loss)
    is_lessequal = bool(epoch_loss >= prev_loss)
    if (is_best):
        prev_loss = epoch_loss
        count = 0
    elif (is_lessequal):
        if (count < 25):
            count = count + 1
        else:
            print("Terminated Training for Early Stopping at Epoch ", epoch+1)
            flag = True

    if flag == False:
#         print(conf_matrix)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': net.state_dict(),
            'class_to_idx': net.class_to_idx,
            'confusion_matrix': conf_matrix
        }, is_best,"./models/carPartModel-"+epoch+".pth")
        # save_checkpoint(net.state_dict(),is_best)
    return flag



/content/Car-damage/car-train-val/splitTrainVal/train/doorScratch/doorScratch-27.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/multipleDamage/multipleDamage-125.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/doorScratch/doorScratch-27.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-24.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/multipleDamage/multipleDamage-125.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-24.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/bumperScratch/bumperScratch-174.jpg
/content/Car-damage/car-train-val/splitTrainVal/train/bumperScratch/bumperScratch-174.jpg
/content/Car-damage/car-train-val/splitTrainVal/train/tailLamp/tailLamp-8.jpg
/content/Car-damage/car-train-val/splitTrainVal/train/tailLamp/tailLamp-8.jpg
/content/Car-damage/car-train-val/splitTrainVal/train/bumperScratch/bumperScratch-144.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/bumpe

TypeError: ignored

/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterFront/glassShatterFront-112.jpg
/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterRear/glassShatterRear-85.jpg
/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterRear/glassShatterRear-85.jpg
/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterRear/glassShatterRear-71.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterRear/glassShatterRear-71.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-143.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/unknown/unknown-143.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/bumperScratch/bumperScratch-107.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/bumperScratch/bumperScratch-107.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/doorDent/doorDent-108.jpeg
/content/Car-damage/car-train-val/splitTrainVal/train/glassShatterRear/glassShatterRear-97.jpeg
/co

In [ ]:
# save_checkpoint
def save_checkpoint(state, is_best, filename):
    if is_best:
        print(state["confusion_matrix"])
        print("=> Saving a new best")
        torch.save(state, filename)
    else:
        print("=> Validation loss did not reduce")

for epoch in range(0, 50):
    flag = False
    train(epoch)
    a = validation(epoch, flag)
    if a == True:
        break

plt.plot(plt_train_loss, label='Training loss')
plt.plot(plt_val_loss, label='Validation loss')
plt.legend(frameon=False)
plt.show()
x = np.arange(0, len(plt_batch_train_loss), len(plt_batch_train_loss)/len(plt_batch_val_loss))
plt.plot(plt_batch_train_loss,label = "Batch train loss")
plt.plot(x,plt_batch_val_loss, label = "Batch validation loss")
plt.legend(frameon=False)
plt.show()
